'''
This is for the EEG Neuroprosthetic Control Dataset

'''

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler


from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt


In [2]:
# TODO: Test how it performs with and without subject ID

def select_feature_columns_exclude_ids(df):
    excluded = {"Subject_ID", "Trial_ID", "Target"}
    features = [c for c in df.columns if c not in excluded and "Wavelet" not in c]
    return features

In [3]:
# Load Data
df = pd.read_csv("C:/Users/ahmad/OneDrive/General/Obsidian2/Classes/Fall 2025/EE595_AppliedMachineLearning/Project/Applied-Machine-Learning-Project/Dataset/Modified/eeg_neuroprosthetic_dataset_first_80.csv")
feature_cols = select_feature_columns_exclude_ids(df)

In [4]:
import plotly.express as px

# 1. Prepare data
# We use df (Original Data) so the values on Y-axis are real/interpretable
plot_df = df.copy() 
plot_df['Target'] = plot_df['Target'].astype(str)
plot_df['Subject_ID'] = plot_df['Subject_ID'].astype(str)

# Melt data for plotting
df_long = plot_df.melt(
    id_vars=['Subject_ID', 'Target', 'Trial_ID'],
    value_vars=feature_cols,
    var_name='Feature',
    value_name='Value'
)

# 2. Create Scatter Plot with Slider (Animation Frame) for Subject
fig = px.scatter(
    df_long,
    x='Feature',
    y='Value',
    color='Target',
    animation_frame='Subject_ID', # Creates the Slider/Play button for Subjects
    hover_data=['Trial_ID'],
    title='Feature Value Distribution (Slide to Change Subject)',
    height=800,
    # Fix Y-axis range so it doesn't jump around when changing subjects
    range_y=[df_long['Value'].min(), df_long['Value'].max()]
)

# 3. Formatting
fig.update_layout(xaxis_tickangle=-90) # Vertical feature names
fig.show()

In [5]:
import plotly.express as px

# 1. Prepare data (using original df for interpretability)
plot_df = df.copy()
plot_df['Target'] = plot_df['Target'].astype(str)

# Melt to long format
df_long = plot_df.melt(
    id_vars=['Target', 'Subject_ID', 'Trial_ID'],
    value_vars=feature_cols,
    var_name='Feature',
    value_name='Value'
)

# 2. Create Box Plot
fig = px.box(
    df_long,
    x='Feature',
    y='Value',
    color='Target', # Different box color for each Target class
    title='Feature Distributions (Box Plots by Target)',
    height=800,
    points="outliers" # Show outlier dots only (or use "all" to see all dots)
)

# 3. Formatting
fig.update_layout(
    xaxis_tickangle=-90, # Vertical feature names
    boxmode='group'      # Group boxes of different targets side-by-side
)

fig.show()

In [6]:
import plotly.express as px

# 1. Prepare data
plot_df = df.copy()
plot_df['Target'] = plot_df['Target'].astype(str)
plot_df['Subject_ID'] = plot_df['Subject_ID'].astype(str)

df_long = plot_df.melt(
    id_vars=['Target', 'Subject_ID', 'Trial_ID'],
    value_vars=feature_cols,
    var_name='Feature',
    value_name='Value'
)

# 2. Create Box Plot with Slider
fig = px.box(
    df_long,
    x='Feature',
    y='Value',
    color='Target',
    animation_frame='Subject_ID', # Adds the Slider for Subject ID
    title='Feature Distributions by Subject (Box Plots)',
    height=800,
    points="outliers",
    # Fix Y-axis range so it doesn't jump when sliding
    range_y=[df_long['Value'].min(), df_long['Value'].max()]
)

# 3. Formatting
fig.update_layout(
    xaxis_tickangle=-90,
    boxmode='group'
)

fig.show()

In [4]:

'''
In an attempt to get the tree to notice subject ID differences
Group by SubjectID and standardize the features per subject
(subtract subject mean, divide by subject std)
'''
subject_means = df.groupby("Subject_ID")[feature_cols].transform("mean")
subject_stds = df.groupby("Subject_ID")[feature_cols].transform("std")

subject_stds = subject_stds.replace(0, 1)
df_scaled = df.copy()
df_scaled[feature_cols] = (df[feature_cols] - subject_means) / subject_stds


print(feature_cols)

# Split data into features and target
X = df_scaled[feature_cols]
y = df_scaled["Target"].astype(int)

# Split training and test data
# Make sure to stratify by target as we don't have much testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

['C3_Optimized_F1', 'C3_Optimized_F2', 'C3_Optimized_F3', 'C3_Optimized_F4', 'C3_Optimized_F5', 'C4_Optimized_F1', 'C4_Optimized_F2', 'C4_Optimized_F3', 'C4_Optimized_F4', 'C4_Optimized_F5', 'Cz_Optimized_F1', 'Cz_Optimized_F2', 'Cz_Optimized_F3', 'Cz_Optimized_F4', 'Cz_Optimized_F5']


In [6]:
# Grid Search
'''
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}
'''

param_grid = {
    'n_estimators': [200],
    'max_depth': [20],
    'min_samples_split': [10],
    "min_samples_leaf": [2]
}



In [7]:
rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

,estimator,RandomForestC...ndom_state=42)
,param_grid,"{'max_depth': [20], 'min_samples_leaf': [2], 'min_samples_split': [10], 'n_estimators': [200]}"
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,200


In [8]:
print("Best parameters found by GridSearchCV:", grid_search.best_params_)
print("Best accuracy found by GridSearchCV:", grid_search.best_score_)

Best parameters found by GridSearchCV: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
Best accuracy found by GridSearchCV: 0.27938184663536775


Best accuracy found by GridSearchCV: 0.29886541471048517

In [9]:
for i in range(3):
    plt.figure(figsize=(20, 10))
    plot_tree(rf.estimators_[i],
              feature_names=X_train.columns,
              filled=True,
              max_depth=2,
              proportion=True)
    plt.title(f'Tree {i+1}')
    plt.show()

AttributeError: 'RandomForestClassifier' object has no attribute 'estimators_'

<Figure size 2000x1000 with 0 Axes>